# VQE in DFT with PsiEmbed and Qiskit

Here we define the inputs as required by PsiEmbed.

In [1]:
from spade import fill_defaults

options = {}
options['geometry'] = """
    O       -1.1867 -0.2472  0.0000
    H       -1.9237  0.3850  0.0000
    H       -0.0227  1.1812  0.8852
    C        0.0000  0.5526  0.0000
    H       -0.0227  1.1812 -0.8852
    C        1.1879 -0.3829  0.0000
    H        2.0985  0.2306  0.0000
    H        1.1184 -1.0093  0.8869
    H        1.1184 -1.0093 -0.8869
    """
options['basis'] = 'cc-pvdz' # basis set 
options['low_level'] = 'b3lyp' # level of theory of the environment 
options['high_level'] = 'mp2' # level of theory of the embedded system
options['n_active_atoms'] = 2 # number of active atoms (first n atoms in the geometry string)
options['low_level_reference'] = 'rohf'
options['high_level_reference'] = 'rohf'
options['package'] = 'pyscf'

keywords = fill_defaults(options)

The first step is to run a mean field caluclation of the whole system.

The Embed class and its subclasses have a method to do this which also sets the following properties:
    Exchange correlation potentials (v_xc_total if embedding potential is not set, or alpha/beta_v_xc_total)
    

In [3]:
from spade import Embed, PySCFEmbed

embed = PySCFEmbed(keywords)

embed.run_mean_field(v_emb=None)


Initialize <pyscf.gto.mole.Mole object at 0x7f52f8533a00> in <pyscf.dft.roks.ROKS object at 0x7f52304457c0>


output file: output.dat


Next we may want to convert to a different basis.

In any case we want to find the orbitals of the active space and environment

In [3]:
embed.__dir__()

['keywords',
 'correlation_energy_shell',
 'shell_size',
 'outfile',
 '_mol',
 '_mean_field',
 'alpha_occupied_orbitals',
 'beta_occupied_orbitals',
 'alpha_j',
 'beta_j',
 'alpha_k',
 'beta_k',
 'alpha_v_xc_total',
 'beta_v_xc_total',
 'alpha',
 '_n_basis_functions',
 'nre',
 'ao_overlap',
 'h_core',
 '__module__',
 '__doc__',
 'run_mean_field',
 'count_active_aos',
 'basis_projection',
 'closed_shell_subsystem',
 'open_shell_subsystem',
 'correlation_energy',
 'effective_virtuals',
 'pseudocanonical',
 'ao_operator',
 '__init__',
 'dot',
 'orbital_rotation',
 'orbital_partition',
 'header',
 'print_scf',
 'print_summary',
 'print_sigma',
 'determinant_overlap',
 'count_shells',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir

In [ ]:
# TODO this first line causes an error cause we have asked for llr to be rohf, which sets 
# alpha/beta_occupied_orbitals but not this generic one. What's going on here?

low_level_orbitals = embed.occupied_orbitals

print(f"{ref.__name__.upper()} Orbitals")
print(low_level_orbitals)

if 'occupied_projection_basis' in keywords:
    projection_orbitals = embed.basis_projection(low_level_orbitals,
        keywords['occupied_projection_basis'])
    n_act_aos = embed.count_active_aos(
        keywords['occupied_projection_basis'])
    ao_overlap = embed.projection_orbitals
else:
    projection_orbitals = low_level_orbitals
    n_act_aos = embed.count_active_aos()
    ao_overlap = embed.ao_overlap

# Orbital rotation and partition into subsystems A and B
rotation_matrix, sigma = embed.orbital_rotation(projection_orbitals,
    n_act_aos, ao_overlap)
n_act_mos, n_env_mos = embed.orbital_partition(sigma)

# Defining active and environment orbitals and density
act_orbitals = low_level_orbitals @ rotation_matrix.T[:, :n_act_mos]
env_orbitals = low_level_orbitals @ rotation_matrix.T[:, n_act_mos:]
act_density = 2.0 * act_orbitals @ act_orbitals.T
env_density = 2.0 * env_orbitals @ env_orbitals.T

We then calclate the cross subsytem terms

In [ ]:
# Retrieving the subsytem energy terms and potential matrices
e_act, e_xc_act, j_act, k_act, v_xc_act = (
    embed.closed_shell_subsystem(act_orbitals))
e_env, e_xc_env, j_env, k_env, v_xc_env = (
    embed.closed_shell_subsystem(env_orbitals))

# Computing cross subsystem terms
j_cross = 0.5 * (matrix_dot(act_density, j_env)
        + matrix_dot(env_density, j_act))

if keywords['package'].lower() == 'psi4':
    k_cross = 0.5 * embed.alpha * (matrix_dot(act_density, k_env)
            + matrix_dot(env_density, k_act))
else:
    k_cross = 0.0

xc_cross = embed.e_xc_total - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross


We can now define the projector used to orthogonalise the Molecular and Atomic orbitals. From this we calculate the embedding potential.

In [ ]:

# TODO: generate orbital (molden) file before embedding

# Defining the embedded core Hamiltonian
projector = keywords['level_shift'] * (embed.ao_overlap @ env_density 
    @ embed.ao_overlap)
v_emb = (j_env - embed.alpha * k_env + embed.v_xc_total - v_xc_act
    + projector)
#h_core_emb = h_core + embedding_potential + projector


Again, we run the mean field method, but with the embedding potential as calulated.

In [ ]:
embed.run_mean_field(v_emb)


In [ ]:
# Overlap between the C_A determinant and the embedded determinant
embed.determinant_overlap(act_orbitals)

# Computing the embedded SCF energy
density_emb = 2.0 * embed.occupied_orbitals @ embed.occupied_orbitals.T
e_act_emb = matrix_dot(density_emb, embed.h_core + 0.5 * embed.j - 0.25 * embed.k)

# Compute the total energy
correction = (matrix_dot(v_emb, density_emb - act_density))
e_mf_emb = e_act_emb + e_env + two_e_cross + embed.nre + correction
embed.print_scf(e_act, e_env, two_e_cross, e_act_emb, correction)



In [ ]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

from embedding_methods import Embed

from qiskit import Aer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.operators import OperatorBase
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.operators import I, X, Z
from qiskit_nature.circuit.library import UCCSD
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit.chemistry import FermionicOperator
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem


/home/miw/miniconda3/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/miw/miniconda3/lib/python3.8/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [ ]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbi